![](https://www.docker.com/sites/default/files/d8/2019-07/Moby-logo.png)

# Dockerfile
---
```
FROM ubuntu:18.04

COPY . /app

RUN make /app

CMD python /app/app.py
```

## COPY vs ADD

1. ADD do additional magic - can read from URL and extract TAR
2. COPY is more straightforward and thus recommended

## CMD vs ENTRYPOINT

1. CMD - default command for `docker run`, easy to override
2. ENTRYPOINT - replaces shell, will process `docker run` params
3. Shell form: `CMD echo "Hello world"`
4. Exec form: `CMD ["echo", "Hello world"]`
5. Exec form is recommended, doesn't require shell.

---
```
# entrypoint with default args:

ENTRYPOINT ["app"]
CMD ["arg1", "arg2"]
```
---
```
# it will ignore CMD
docker run arg3
```

# Effective build cache

Most stable commands on top, often changing on bottom.

```
...

COPY requirements/prod.txt /requirements.txt
RUN pip install -r /requirements.txt

COPY . /app

...
```

# Build context and .dockerignore

1. All files from dir where `docker build` is run are send to docker process.
2. .dockerignore (syntax as .gitignore) allows to exclude some files.
3. It makes build faster and Dockerfile simpler, e.g. `COPY . /app`

# Layers

1. Every command in Dockerfile creates layer.
2. Layer is just a files diff from layer below.
3. Layers are read-only which makes them reusable.
4. Copy-on-Write (CoW) strategy - copy then modify.
5. `docker history <image>` - shows image's layers.

![](img/sharing-layers.jpg)

## Storage driver

1. Determines how layers are stored and applied.
2. Default `overlay2` operates on file level.
3. There are also block level drivers, e.g. `btrfs`.
4. File level is simpler and memory efficient, but uses more storage.
5. Block level is better for write-heavy workloads.
6. File level + volumes are the way to go.

## How to get to layer's files?
---
```
docker image ls
docker history <image>
docker inspect <layer>
cd GraphDriver -> Data -> UpperDir
```

# Container vs image

1. Image is a sequence of read-only layers.
2. Container is a writable layer on top of image.
3. Container can be saved as layer by `docker commit`.

![](img/container-layers.jpg)

# Volumes

1. Mounts outside dir to path inside container.
2. Volume - dir created and managed by Docker, typically in /var/lib/docker
3. Bind mounts - dir from host machine, e.g. `./`
4. All container's persistent data should be stored in volume.
5. Bind mounts are great for local development.

---
```
> docker-compose.yml

services:
  db:
    volumes:
      - db-data:/var/lib/postgresql/data
  app:
    volumes:
      - .:/app

volumes:
  db-data:
```

![](img/volumes.png)

# Docker Compose

1. Creates network between defined services, separated from localhost.
2. Service's names are hosts in this internal network.
3. Allows to define dependencies between containers.
4. Automatically load `.env` file.

# Troubleshooting
---
```
# it's like 42
docker inspect <hash>

# no comment
docker logs --tail=50 -f <container>
docker-compose logs --tail=50 -f <service>

# run image with bash instead of default command
docker run --rm -it <image> bash
docker-compose run <service> bash

# run image with bash instead of default entrypoint
docker run --rm -it --entrypoint bash <image>
docker-compose run --entrypoint bash <service>

# enter to running container with bash
docker exec -it <container> bash
docker-compose exec <service> bash
```

# Docker and Linux kernel

1. `cgroups` - control groups - allow to manage resources per process.
2. `namespaces` - allow to isolate process groups from each other, e.g. `pid`, `usr`

## Namespaces

1. Namespaces uses IDs from outside normal range (usually 65535)
2. IDs inside namespace are mapped e.g. `uid 65536 -> 1000`
3. Errors saying that ID > 65535 is unknown are probably problem with namespaces.
4. Bitbucket often has such errors ;)

![](img/kernel_communication.png)

![](img/end.jpg)